In [1]:
import torch
from torch import nn
from tqdm.auto import tqdm
from torchvision import transforms, utils
from torchvision.datasets import MNIST
from torch.utils.data import TensorDataset
from torchvision.utils import make_grid
from torch.utils.data import Dataset, DataLoader
from torch.utils.data import random_split
import torch.nn.functional as F


import matplotlib.pyplot as plt
from sklearn.preprocessing import scale as standard_scale
from sklearn.model_selection import train_test_split
from skorch.helper import predefined_split
from skorch.callbacks import LRScheduler
from skorch.dataset import ValidSplit

# torch.manual_seed(0) # Set for our testing purposes, please do not change!

from braindecode.datasets import MOABBDataset
from braindecode.preprocessing import (
    exponential_moving_standardize, preprocess, Preprocessor)
from braindecode.preprocessing import \
    create_windows_from_events, create_fixed_length_windows
from braindecode.models import EEGNetv4
from braindecode import EEGClassifier
import mne

import pandas as pd
import numpy as np

/usr/local/lib/python3.11/dist-packages/tqdm/auto.py:21: TqdmWarning: IProgress not found. Please update jupyter and ipywidgets. See https://ipywidgets.readthedocs.io/en/stable/user_install.html
  from .autonotebook import tqdm as notebook_tqdm
2023-11-17 18:23:16.440654: E tensorflow/compiler/xla/stream_executor/cuda/cuda_dnn.cc:9342] Unable to register cuDNN factory: Attempting to register factory for plugin cuDNN when one has already been registered
2023-11-17 18:23:16.440744: E tensorflow/compiler/xla/stream_executor/cuda/cuda_fft.cc:609] Unable to register cuFFT factory: Attempting to register factory for plugin cuFFT when one has already been registered
2023-11-17 18:23:16.440805: E tensorflow/compiler/xla/stream_executor/cuda/cuda_blas.cc:1518] Unable to register cuBLAS factory: Attempting to register factory for plugin cuBLAS when one has already been registered
2023-11-17 18:23:16.447651: I tensorflow/core/platform/cpu_feature_guard.cc:182] This TensorFlow binary is optimized 

Tensorflow not install, you could not use those pipelines


# Classe Gerador

In [2]:
class Generator(nn.Module):
    '''
    Generator Class
    Values:
        input_dim: the dimension of the input vector, a scalar
        im_chan: the number of channels of the output eeg, a scalar
        hidden_dim: the inner dimension, a scalar
    '''
    def __init__(self, input_dim=68, im_chan=1, hidden_dim=64):
        super(Generator, self).__init__()
        self.input_dim = input_dim
        # Build the neural network
        self.gen = nn.Sequential(
            #### For 3 channels
#             self.make_gen_block(input_dim, hidden_dim * 4,      kernel_size = (1,60), stride = (1,1)),
#             self.make_gen_block(hidden_dim * 4, hidden_dim * 2, kernel_size = (1,60), stride = (1,1)),
#             self.make_gen_block(hidden_dim * 2, hidden_dim,     kernel_size = (1,60), stride = (1,1)),
#             self.make_gen_block(hidden_dim, im_chan,            kernel_size = (3,50), stride = (1,2), padding = (0,2), final_layer=True),
            #### For 22 channels
            self.make_gen_block(input_dim, hidden_dim * 4,      kernel_size = (3,60), stride = (1,1)),
            self.make_gen_block(hidden_dim * 4, hidden_dim * 2, kernel_size = (4,60), stride = (3,1)),
            self.make_gen_block(hidden_dim * 2, hidden_dim,     kernel_size = (3,60), stride = (2,1)),
            self.make_gen_block(hidden_dim, im_chan,            kernel_size = (2,50), stride = (1,2), padding = (0,2), final_layer=True),
        )

    def make_gen_block(self, input_channels, output_channels, kernel_size, stride, padding = 0, final_layer=False):
        '''
        Function to return a sequence of operations corresponding to a generator block of DCGAN;
        a transposed convolution, a batchnorm (except in the final layer), and an activation.
        Parameters:
            input_channels: how many channels the input feature representation has
            output_channels: how many channels the output feature representation should have
            kernel_size: the size of each convolutional filter, equivalent to (kernel_size, kernel_size)
            stride: the stride of the convolution
            final_layer: a boolean, true if it is the final layer and false otherwise 
                      (affects activation and batchnorm)
        '''
        if not final_layer:
            return nn.Sequential(
                nn.ConvTranspose2d(input_channels, output_channels, kernel_size, stride, padding),
                nn.BatchNorm2d(output_channels),
                nn.ReLU(inplace=True),
            )
        else:
            return nn.Sequential(
                nn.ConvTranspose2d(input_channels, output_channels, kernel_size, stride, padding),
                nn.Tanh(),
            )

    def forward(self, noise):
        '''
        Function for completing a forward pass of the generator: Given a noise tensor, 
        returns generated images.
        Parameters:
            noise: a noise tensor with dimensions (n_samples, input_dim)
        '''
        x = noise.view(len(noise), self.input_dim, 1, 1)
        return self.gen(x)

def get_noise(n_samples, input_dim, device='cpu'):
    '''
    Function for creating noise vectors: Given the dimensions (n_samples, input_dim)
    creates a tensor of that shape filled with random numbers from the normal distribution.
    Parameters:
        n_samples: the number of samples to generate, a scalar
        input_dim: the dimension of the input vector, a scalar
        device: the device type
    '''
    return torch.randn(n_samples, input_dim, device=device)


# Função para gerar dados

In [9]:
def get_one_hot_labels(labels, n_classes):
    '''
    Function for creating one-hot vectors for the labels, returns a tensor of shape (?, num_classes).
    Parameters:
        labels: tensor of labels from the dataloader, size (?)
        n_classes: the total number of classes in the dataset, an integer scalar
    '''
    return F.one_hot(labels,n_classes)

def combine_vectors(x, y):
    '''
    Function for combining two vectors with shapes (n_samples, ?) and (n_samples, ?).
    Parameters:
      x: (n_samples, ?) the first vector. 
        In this assignment, this will be the noise vector of shape (n_samples, z_dim), 
        but you shouldn't need to know the second dimension's size.
      y: (n_samples, ?) the second vector.
        Once again, in this assignment this will be the one-hot class vector 
        with the shape (n_samples, n_classes), but you shouldn't assume this in your code.
    '''
    combined = torch.cat((x.float(),y.float()), 1)
    return combined


def generate_samples_with_labels(label, n_samples, generator, z_dim = 64, channel = None, extra_dim = True):
    '''
    Function for generating samples, once the generator has been trained
        label: label of the movement to be sampled. See dictionary below
        {'feet': 0, 'left_hand': 1, 'right_hand': 2, 'tongue': 3}
        n_samples: number of samples to be generated
        channel: electrode {'C3': 0, 'Cz': 1, 'C4': 2} -> Default: All channels
        generator: the trained generator
    '''

    n_classes = 4

    if channel == None:
        noise_4_gen = get_noise(n_samples, z_dim)
        label = get_one_hot_labels(torch.Tensor([label]).long(), n_classes).repeat(n_samples,1)

        noise_and_labels = combine_vectors(noise_4_gen, label)
        fake = generator(noise_and_labels)

        if extra_dim == False:
            fake = fake.reshape((fake.shape[0], fake.shape[2], fake.shape[3]))
        return fake
    else:
        noise_4_gen = get_noise(n_samples, z_dim)
        label = get_one_hot_labels(torch.Tensor([label]).long(), n_classes).repeat(n_samples,1)

        noise_and_labels = combine_vectors(noise_4_gen, label)
        fake = generator(noise_and_labels)
        filtered_channel_fake = torch.select(fake, dim = 2, index = channel)

        if extra_dim == False:
            filtered_channel_fake = filtered_channel_fake.reshape((filtered_channel_fake.shape[0], filtered_channel_fake.shape[2]))

        return filtered_channel_fake


# Configuração

In [7]:
PATH = "./gen_22ch_batch64_lr37e6_decay7.pt"
gen = Generator()
gen.load_state_dict(torch.load(PATH))

<All keys matched successfully>

In [35]:
tamanho_sample = 10 # Deixar 288

gen_data_label0 = generate_samples_with_labels(generator = gen, label = 0,n_samples = tamanho_sample)
gen_data_label1 = generate_samples_with_labels(generator = gen, label = 1,n_samples = tamanho_sample)
gen_data_label2 = generate_samples_with_labels(generator = gen, label = 2,n_samples = tamanho_sample)
gen_data_label3 = generate_samples_with_labels(generator = gen, label = 3,n_samples = tamanho_sample)

# Falta concatenar

In [41]:
gen_data_label3.shape

torch.Size([10, 1, 22, 400])

In [45]:
# Concat
gen_data_final = torch.cat((gen_data_label0,
                          gen_data_label1,
                          gen_data_label2,
                          gen_data_label3), dim = 0)

In [48]:
gen_data_final.shape

torch.Size([40, 1, 22, 400])